In [3]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [4]:
import gradio as gr 

In [6]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
openrouter_api_key=os.getenv('OPENROUTER_API_KEY')
google_api_key = os.getenv('GEMINI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if openrouter_api_key:
    print(f"OpenRouter API Key exists and begins {openrouter_api_key[:3]}")
else:
    print("OpenRouter API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
OpenRouter API Key exists and begins sk-
Google API Key exists and begins AIzaSyBQ


In [7]:
openai = OpenAI()

openrouter_url = "https://openrouter.ai/api/v1"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

openrouter = OpenAI(base_url=openrouter_url, api_key=openrouter_api_key)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [8]:
system_message = "You are a helpful assistant"

def message_gpt(prompt):
    messages = [{"role": "system", "content": system_message}, {"role": "user", "content": prompt}]
    response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
    return response.choices[0].message.content

In [9]:
message_gpt("What is today's date?")

"Today's date is June 15, 2024."

In [10]:
def shout(text):
    print(f"Shout has been called with input {text}")
    return text.upper()

In [11]:
shout("hello")

Shout has been called with input hello


'HELLO'

In [ ]:
shout("hello")
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch()

Shout has been called with input hello
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Shout has been called with input hello
Shout has been called with input none


In [13]:
system_message = "You are a helpful assistant that responds in markdown without code blocks"

message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


In [14]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [15]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.


In [17]:
def stream_openrouter(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openrouter.chat.completions.create(
        model='qwen/qwen3-235b-a22b:free',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [18]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for Claude 4.5 Sonnet", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_openrouter,
    title="Claude", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


In [19]:
def stream_model(prompt, model):
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="openroter":
        result = stream_openrouter(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [20]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for the LLM", lines=7)
model_selector = gr.Dropdown(["GPT", "openroter"], label="Select model", value="GPT")
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_model,
    title="LLMs", 
    inputs=[message_input, model_selector], 
    outputs=[message_output], 
    examples=[
            ["Explain the Transformer architecture to a layperson", "GPT"],
            ["Explain the Transformer architecture to an aspiring AI engineer", "openroter"]
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.
